##Machine translation

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import model_selection
import re
import tqdm

In [2]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download -d lonnieqin/englishspanish-translation-dataset

  0% 0.00/2.72M [00:00<?, ?B/s]
100% 2.72M/2.72M [00:00<00:00, 181MB/s]


In [4]:
!unzip /content/englishspanish-translation-dataset.zip

Archive:  /content/englishspanish-translation-dataset.zip
  inflating: data.csv                


In [26]:
data_df=pd.read_csv('/content/englishspanish-translation-dataset.zip')
data_df


,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
...,...,...
118959,There are four main causes of alcohol-related ...,Hay cuatro causas principales de muertes relac...
118960,There are mothers and fathers who will lie awa...,Hay madres y padres que se quedan despiertos d...
118961,A carbon footprint is the amount of carbon dio...,Una huella de carbono es la cantidad de contam...
118962,Since there are usually multiple websites on a...,Como suele haber varias páginas web sobre cual...


In [6]:
def clean_data(text):
  text = text.lower()
  text = re.sub('\[.*?\]',' ', text)
  text = re.sub('https?://S+|www\.\S+','', text)
  text = re.sub('<.*?>+','', text)
  text = re.sub('\n','', text)
  text = re.sub(r'[^\w\s]','', text)
  text = re.sub('\w*\d\w*','', text)
  return text

data_df.english = data_df.english.map(clean_data)
data_df.spanish = data_df.spanish.map(clean_data)
data_df

,english,spanish
0,go,ve
1,go,vete
2,go,vaya
3,go,váyase
4,hi,hola
...,...,...
118959,there are four main causes of alcoholrelated d...,hay cuatro causas principales de muertes relac...
118960,there are mothers and fathers who will lie awa...,hay madres y padres que se quedan despiertos d...
118961,a carbon footprint is the amount of carbon dio...,una huella de carbono es la cantidad de contam...
118962,since there are usually multiple websites on a...,como suele haber varias páginas web sobre cual...


In [7]:
def add_start_end(text):
  text = f'<start> {text} <end>'
  return text

data_df.english = data_df.english.map(add_start_end)
data_df.spanish = data_df.spanish.map(add_start_end)

In [8]:
data_df

,english,spanish
0,<start> go <end>,<start> ve <end>
1,<start> go <end>,<start> vete <end>
2,<start> go <end>,<start> vaya <end>
3,<start> go <end>,<start> váyase <end>
4,<start> hi <end>,<start> hola <end>
...,...,...
118959,<start> there are four main causes of alcoholr...,<start> hay cuatro causas principales de muert...
118960,<start> there are mothers and fathers who will...,<start> hay madres y padres que se quedan desp...
118961,<start> a carbon footprint is the amount of ca...,<start> una huella de carbono es la cantidad d...
118962,<start> since there are usually multiple websi...,<start> como suele haber varias páginas web so...


In [9]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;= ?@[\\]^_`{|}~\t\n',oov_token='<oov>')
  lang_tokenizer.fit_on_texts(lang)
  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
  return tensor, lang_tokenizer


In [10]:
eng_sequences, eng_tokenizer= tokenize(data_df.english)
spn_sequences, spn_tokenizer= tokenize(data_df.spanish)

In [11]:
x_train, x_test, y_train, y_test= model_selection.train_test_split(eng_sequences, spn_sequences, test_size= 0.1, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((107067, 49), (11897, 49), (107067, 51), (11897, 51))

In [12]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print('%d ----> %s'% (t,lang.index_word[t]))

print('English:')
convert(eng_tokenizer,x_train[0] )

print('\nSpanish:')
convert(spn_tokenizer,x_train[0] )

English:
2 ----> <start>
48 ----> there
54 ----> were
140 ----> two
1489 ----> pieces
13 ----> of
487 ----> cake
3 ----> <end>

Spanish:
2 ----> <start>
48 ----> si
54 ----> algo
140 ----> sin
1489 ----> pregunté
13 ----> un
487 ----> perdido
3 ----> <end>


In [13]:
vocab_inp_size = len(eng_tokenizer.word_index)+1
vocab_tar_size = len(spn_tokenizer.word_index)+1
embedding_dim = 256
units = 1024
batch_size = 32

In [27]:
def create_dataset(x, y, batch_size=32):
  data = tf.data.Dataset.from_tensor_slices((x,y))
  data = data.shuffle(1024)
  data = data.batch(batch_size, drop_remainder=True)
  data = data.prefetch(tf.data.experimental.AUTOTUNE)
  return data

train_dataset = create_dataset(x_train, y_train)
test_dataset = create_dataset(x_test, y_test)



In [15]:
for eng, spn in train_dataset.take(1):
  print(f'English:{eng.shape}\n{eng}')

  print(f'\nSpanish:{spn.shape}\n{spn}')

English:(32, 49)
[[  2   5 291 ...   0   0   0]
 [  2   8  10 ...   0   0   0]
 [  2 106 272 ...   0   0   0]
 ...
 [  2   5 151 ...   0   0   0]
 [  2   8 156 ...   0   0   0]
 [  2  27 204 ...   0   0   0]]

Spanish:(32, 51)
[[  2 277   5 ...   0   0   0]
 [  2   8  12 ...   0   0   0]
 [  2  71 221 ...   0   0   0]
 ...
 [  2  35   9 ...   0   0   0]
 [  2   8 128 ...   0   0   0]
 [  2  20 168 ...   0   0   0]]


In [16]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size) :
    super(Encoder, self).__init__()

    self.batch_size = batch_size
    self.encoder_units = encoder_units
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim, mask_zero=True)
    self.gru = tf.keras.layers.GRU(self.encoder_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def  call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x,initial_state= hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_size, self.encoder_units))

In [17]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, decoder_units, batch_size) :
    super(Decoder, self).__init__()

    self.batch_size = batch_size
    self.decoder_units = decoder_units
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim, mask_zero=True)
    self.gru = tf.keras.layers.GRU(self.decoder_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    x = self.embedding(x)
    output, hidden = self.gru(x, initial_state = hidden)
    output = tf.reshape(output, (-1, output.shape[2]))
    x =  tf.nn.softmax(self.fc(output))
    return x, hidden


In [18]:
encoder = Encoder(vocab_inp_size, embedding_dim,units, batch_size)
sample_hidden_state = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(eng, sample_hidden_state)
print(f'Encoder output shape(batch size, sequence length, units) {sample_output.shape}')
print(f'Encoder hidden state shape(batch size, sequence length, units) {sample_output.shape}')

Encoder output shape(batch size, sequence length, units) (32, 49, 1024)
Encoder hidden state shape(batch size, sequence length, units) (32, 49, 1024)


In [19]:
decoder = Decoder(vocab_tar_size, embedding_dim,units, batch_size)

sample_decoder_output, _ = decoder(tf.random.uniform((batch_size, 1)), sample_hidden)
print(f'Decoder output shape(batch size, sequence length, units) {sample_output.shape}')

Decoder output shape(batch size, sequence length, units) (32, 49, 1024)


#Create the Optimizer and the Loss Function

In [20]:
# create the optimizer using the Adam optimizer
optimizer = tf.keras.optimizers.Adam()
# create the loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction='none')

# define the loss function for the training
def loss_function(real, pred):
  # create the mask to ignore the padding tokens
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  # mask shape == (batch_size, sequence_length)
  # calculate the loss
  loss_ = loss_object(real, pred)
  # mask the loss
  # how the mask works:
  # if the value is 1, the loss is calculated
  # if the value is 0, the loss is ignored
    #[1,1,1,1,1,1,0,0,0,0,0] mask
    # *
    #[2,6,2,1,6,3,2,1,5,7,9] input
    # =
    #[2,6,2,1,6,3,0,0,0,0,0] output
  mask = tf.cast(mask, dtype=loss_.dtype)
  # mask shape == (batch_size, sequence_length)

  loss_ *= mask
  # calculate the average loss per batch
  return tf.reduce_mean(loss_)

In [21]:
# create the training metric
train_loss = tf.metrics.Mean(name='train loss')
# create the testing metric
test_loss =tf.metrics.Mean(name='test loss')

Create the Training Step

Computing Gradients Using Autodiff

In [22]:
# create the training step
# using the tf.function decorator to speed up the training process by converting the training function to a TensorFlow graph
@tf.function
# define the training step
def train_step(inputs, target, enc_hidden):
  # the encoder_hidden is the initial hidden state of the encoder
  # enc_hidden shape == (batch_size, hidden_size)

  # inilaize the loss to zero
  loss = 0
  # create the gradient tape to record the gradient of the loss with respect to the weights

  with tf.GradientTape() as tape:
    # pass the input to the encoder
    # enc_output shape == (batch_size, 49, hidden_size)
    # enc_hidden shape == (batch_size, hidden_size)
    # using the encoder to get the encoder_output and the encoder_hidden
    # using the encoder_hidden as the initial hidden state of the decoder
    enc_output, enc_hidden = encoder(inputs, enc_hidden)
    # set the initial decoder hidden state to the encoder hidden state
    dec_hidden = enc_hidden

    # create the start token
    # start_token shape == (batch_size, 1)
    # repeat the start token for the batch size times
    dec_input = tf.expand_dims([spn_tokenizer.word_index['<start>']] * inputs.shape[0], 1)

    # Teacher forcing - feeding the target as the next input

    for t in range(1, target.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)
      # calculate the loss for the current time step using the loss function
      loss += loss_function(target[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(target[:, t], 1)
  # calculate the loss for the current batch
  batch_loss = (loss / int(target.shape[1]))

  # get the trainable variables
  variables = encoder.trainable_variables + decoder.trainable_variables
  # calculate the gradients using the tape
  gradients = tape.gradient(loss, variables)
  # update the trainable variables
  optimizer.apply_gradients(zip(gradients, variables))
  # add the loss to the training loss metric
  train_loss(batch_loss)
  return batch_loss

In [23]:
# create the training step
# using the tf.function decorator to speed up the training process by converting the training function to a TensorFlow graph
@tf.function
def test_step(inputs, target, enc_hidden):
    # the encoder_hidden is the initial hidden state of the encoder
    # enc_hidden shape == (batch_size, hidden_size)
    # inilaize the loss to zero
    loss = 0
    # pass the input to the encoder
    # enc_output shape == (batch_size, 49, hidden_size)
    # enc_hidden shape == (batch_size, hidden_size)
    # using the encoder to get the encoder_output and the encoder_hidden
    enc_output, enc_hidden = encoder(inputs, enc_hidden)
    # set the initial decoder hidden state to the encoder hidden state
    dec_hidden = enc_hidden
    # create the start token
    # start_token shape == (batch_size, 1)
    # repeat the start token for the batch size times
    dec_input = tf.expand_dims([spn_tokenizer.word_index['<start>']] * inputs.shape[0], 1)
    for t in range(1, target.shape[1]):
        # passing enc_output to the decoder with dec_hidden as the initial hidden state
        predictions, dec_hidden = decoder(dec_input, dec_hidden)
        # calculate the loss for the current time step using the loss function
        loss += loss_function(target[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(target[:, t], 1)
    # calculate the loss for the current batch
    batch_loss = (loss / int(target.shape[1]))
    # add the batch loss to the test loss metric
    test_loss(batch_loss)

In [24]:
# set the epochs to 3
EPOCHS = 3
# set the old test loss to high number
old_test_loss=1000000
# create the training loop
for epoch in range(EPOCHS):
    # reset the training loss metric
    train_loss.reset_states()
    # reset the testing loss metric
    test_loss.reset_states()

    # initalize the hidden state of the encoder to zeros
    enc_hidden = encoder.initialize_hidden_state()
    # create the training progress bar set the total number of batches to the length of the training dataset and the batch size to the test size
    steps_per_epoch = eng_sequences.shape[0]//batch_size #=> 3717 batch in the dataset
    bar = tf.keras.utils.Progbar(target=steps_per_epoch)

    count=0
    # iterate over the training dataset
    for (batch, (inputs, target)) in enumerate(train_dataset):
        # update the progress bar
        count += 1
        # run the training step
        batch_loss = train_step(inputs, target, enc_hidden)
        bar.update(count)  # manually update the progress bar




    # iterate over the testing dataset
    for (batch, (inputs, target)) in enumerate(test_dataset):
        count += 1
        # run the testing step
        batch_loss = test_step(inputs, target, enc_hidden)
        bar.update(count)
    # save the best performance model on the test dataset

    if old_test_loss> test_loss.result():
        # set the old test loss to the test loss
        old_test_loss= test_loss.result()
        encoder.save(filepath='/content/models/encoder')
        decoder.save(filepath='/content/models/decoder')
        print('Model is saved')
    # print the training and testing loss
    print('#' * 50)
    print(f'Epoch #{epoch + 1}')
    print(f'Training Loss {train_loss.result()}')
    print(f'Testing Loss {test_loss.result()}')
    print('#' * 50)

3716/3717 [============================>.] - ETA: 0sModel is saved
##################################################
Epoch #1
Training Loss 0.5750858783721924
Testing Loss 0.4082885980606079
##################################################
3716/3717 [============================>.] - ETA: 0sModel is saved
##################################################
Epoch #2
Training Loss 0.30662375688552856
Testing Loss 0.3342507481575012
##################################################
3716/3717 [============================>.] - ETA: 0sModel is saved
##################################################
Epoch #3
Training Loss 0.18461808562278748
Testing Loss 0.33035048842430115
##################################################


In [29]:
#  create the translate function
# the translate function takes in the english as input and translation the sentence to spanish
def translate(sentence):

  # clean the input english sentence
  sentence = clean_data(sentence)
  # add the start token to the sentence
  sentence =add_start_end(sentence)
  # tokenize the sentence
  inputs = eng_tokenizer.texts_to_sequences([sentence])
  # pad the sentence
  inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                         maxlen=49,
                                                         padding='post')

  # initalize the hidden state of the encoder to zeros
  hidden = [tf.zeros((1, units))]
  # pass the sentence to the encoder with the hidden state as the initial hidden state
  enc_out, enc_hidden = encoder(inputs, hidden)
  # set the initial decoder hidden state to the encoder hidden state
  dec_hidden = enc_hidden
  # create the start token
  # start_token shape == (batch_size, 1)
  # repeat the start token for the batch size times
  dec_input = tf.expand_dims([spn_tokenizer.word_index['<start>']], 0)
  # create the result string
  result = ''
  # loop over the length of the sentence (51)

  for t in range(51):
    # passing the encoder output and the decoder hidden state to the decoder make sure the decoder input is the previous predicted word
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # getting the predicted word index
    predicted_id = tf.argmax(predictions[0]).numpy()
    # getting the predicted word using the predicted index
    # add the predicted word to the result string
    result += spn_tokenizer.index_word[predicted_id] + ' '
    # if the predicted word is the <end> token then stop the loop
    if spn_tokenizer.index_word[predicted_id] == '<end>':
      # remove the <start> and <end> tokens from the result string
      result = result.replace('<start> ', '')
      result = result.replace(' <end> ','')
      # remove the <start> and <end> tokens from the sentence string
      sentence = sentence.replace('<start> ', '')
      sentence = sentence.replace(' <end>', '')
      return  sentence, result

    # using the predicted word as the next decoder input
    dec_input = tf.expand_dims([predicted_id], 0)
  # remove the <start> and <end> tokens from the result string
  result = result.replace('<start> ', '')
  result = result.replace(' <end> ','')
  # remove the <start> and <end> tokens from the sentence string
  sentence = sentence.replace('<start> ', '')
  sentence = sentence.replace(' <end>', '')





  # return the result string and the original sentence
  return sentence, result

In [35]:
translate('i am a boy ')

('i am a boy ', 'soy un chico')